In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline

# Add line profiler
%load_ext line_profiler

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize

from IPython.display import display, HTML

from enum import Enum
import subprocess
import os
import gc
import datetime
import importlib
import logging
import netCDF4

import pycuda.driver as cuda
import pycuda.compiler

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

from GPUSimulators import Common, IPythonMagic

In [ ]:
%setup_logging --out test_schemes.log

In [ ]:
%cuda_context_handler --no_autotuning my_context 

In [ ]:
#Set large figure sizes as default
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')
rc('animation', bitrate=1800)

In [ ]:
def genShockBubble(nx, ny, gamma):
    width = 4.0
    height = 1.0
    dx = width / float(nx)
    dy = height / float(ny)

    x_center = 0.5 #dx*nx/2.0
    y_center = 0.5 #dy*ny/2.0


    rho = np.ones((ny, nx), dtype=np.float32)
    u = np.zeros((ny, nx), dtype=np.float32)
    v = np.zeros((ny, nx), dtype=np.float32)
    E = np.zeros((ny, nx), dtype=np.float32)
    p = np.ones((ny, nx), dtype=np.float32)

    x = (dx*(np.arange(0, nx, dtype=np.float32)+0.5) - x_center)
    y = (dy*(np.arange(0, ny, dtype=np.float32)+0.5) - y_center)
    xv, yv = np.meshgrid(x, y, sparse=False, indexing='xy')
       
    #Bubble
    radius = 0.25
    bubble = np.sqrt(xv**2+yv**2) <= radius
    rho = np.where(bubble, 0.1, rho)
    
    #Left boundary
    left = (xv - xv.min() < 0.1)
    rho = np.where(left, 3.81250, rho)
    u = np.where(left, 2.57669, u)
    
    #Energy
    p = np.where(left, 10.0, p)
    E = 0.5*rho*(u**2+v**2) + p/(gamma-1.0)
    
    #Estimate dt
    scale = 0.9
    max_rho_estimate = 5.0
    max_u_estimate = 5.0
    dx = width/nx
    dy = height/ny
    dt = scale * min(dx, dy) / (max_u_estimate + np.sqrt(gamma*max_rho_estimate))

    return rho, rho*u, rho*v, E, dx, dy, dt

nx = 400
rho0, rho_u0, rho_v0, E0, dx, dy, dt = genShockBubble(nx, nx//4, 1.4)
plt.figure()
plt.imshow(rho0)
plt.colorbar(orientation='vertical', aspect=20, pad=0.02, shrink=0.3)

In [ ]:
def genKelvinHelmholtz(nx, ny, gamma, roughness=0.5):
    """
    Roughness parameter in (0, 1.0] determines how squiggly the interface betweeen the zones is
    """
    
    def genZones(nx, ny, n):
        """
        Generates the zones of the two fluids of K-H
        """
        zone = np.zeros((ny, nx), dtype=np.int32)

        dx = 1.0 / nx
        dy = 1.0 / ny

        def genSmoothRandom(nx, n):
            assert (n <= nx), "Number of generated points nx must be larger than n"
            
            if n == nx:
                return np.random.random(nx)-0.5
            else:
                from scipy.interpolate import interp1d

                #Control points and interpolator
                xp = np.linspace(0.0, 1.0, n)
                yp = np.random.random(n) - 0.5
                f = interp1d(xp, yp, kind='cubic')

                #Interpolation points
                x = np.linspace(0.0, 1.0, nx)
                return f(x)



        x = np.linspace(0, 1, nx)
        y = np.linspace(0, 1, ny)

        _, y = np.meshgrid(x, y)

        #print(y+a[0])

        a = genSmoothRandom(nx, n)*dy
        zone = np.where(y > 0.25+a, zone, 1)

        a = genSmoothRandom(nx, n)*dy
        zone = np.where(y < 0.75+a, zone, 1)
        
        return zone
        
    width = 2.0
    height = 1.0
    dx = width / float(nx)
    dy = height / float(ny)

    rho = np.empty((ny, nx), dtype=np.float32)
    u = np.empty((ny, nx), dtype=np.float32)
    v = np.zeros((ny, nx), dtype=np.float32)
    p = 2.5*np.ones((ny, nx), dtype=np.float32)

    #Generate the different zones    
    zones = genZones(nx, ny, max(1, min(nx, int(nx*roughness))))
    
    #Zone 0
    zone0 = zones == 0
    rho = np.where(zone0, 1.0, rho)
    u = np.where(zone0, 0.5, u)
    
    #Zone 1
    zone1 = zones == 1
    rho = np.where(zone1, 2.0, rho)
    u = np.where(zone1, -0.5, u)
    
    E = 0.5*rho*(u**2+v**2) + p/(gamma-1.0)
    
    #Estimate dt
    scale = 0.9
    max_rho_estimate = 3.0
    max_u_estimate = 2.0
    dx = width/nx
    dy = height/ny
    dt = scale * min(dx, dy) / (max_u_estimate + np.sqrt(gamma*max_rho_estimate))

    return rho, rho*u, rho*v, E, dx, dy, dt

nx = 400
rho0, rho_u0, rho_v0, E0, dx, dy, dt = genKelvinHelmholtz(nx, nx//4, 1.4, 0.25)
plt.figure()
plt.imshow(rho0)
plt.colorbar(orientation='vertical', aspect=20, pad=0.02, shrink=0.3)

In [ ]:
def genRayleighTaylor(nx, ny, gamma):
    width = 0.5
    height = 1.5
    dx = width / float(nx)
    dy = height / float(ny)
    g = 0.1

    rho = np.zeros((ny, nx), dtype=np.float32)
    u = np.zeros((ny, nx), dtype=np.float32)
    v = np.zeros((ny, nx), dtype=np.float32)
    p = np.zeros((ny, nx), dtype=np.float32)
    
    x = dx*np.arange(0, nx, dtype=np.float32)-width*0.5
    y = dy*np.arange(0, ny, dtype=np.float32)-height*0.5
    xv, yv = np.meshgrid(x, y, sparse=False, indexing='xy')
    
    #y_threshold = -0.01*np.cos(2*np.pi*np.abs(x)/0.5)
    #rho = np.where(yv <= y_threshold, 1.0, rho)
    #rho = np.where(yv > y_threshold, 2.0, rho)
    
    rho = np.where(yv <= 0.0, 1.0, rho)
    rho = np.where(yv > 0.0, 2.0, rho)
    
    v = 0.01*(1.0 + np.cos(2*np.pi*xv/0.5))/4
    
    p = 2.5 - rho*g*yv
    E = 0.5*rho*(u**2+v**2) + p/(gamma-1.0)
    
    #Estimate dt
    scale = 0.9
    max_rho_estimate = 3.0
    max_u_estimate = 1.0
    dx = width/nx
    dy = height/ny
    dt = scale * min(dx, dy) / (max_u_estimate + np.sqrt(gamma*max_rho_estimate))

    return rho, rho*u, rho*v, E, dx, dy, dt

nx = 400
rho0, rho_u0, rho_v0, E0, dx, dy, dt = genRayleighTaylor(nx, nx*3, 1.4)
plt.figure()
plt.subplot(1,4,1)
plt.imshow(rho0, origin='bottom')
plt.colorbar(orientation='horizontal', aspect=20, pad=0.02, shrink=0.9)

plt.subplot(1,4,2)
plt.imshow(rho_u0, origin='bottom')
plt.colorbar(orientation='horizontal', aspect=20, pad=0.02, shrink=0.9)

plt.subplot(1,4,3)
plt.imshow(rho_v0, origin='bottom')
plt.colorbar(orientation='horizontal', aspect=20, pad=0.02, shrink=0.9)

plt.subplot(1,4,4)
plt.imshow(E0, origin='bottom')
plt.colorbar(orientation='horizontal', aspect=20, pad=0.02, shrink=0.9)

In [ ]:
def makeSimulation(outfile, t_end, rho0, rho_u0, rho_v0, E0, nx, ny, dx, dy, dt, gamma):
    #Construct simulator
    arguments = {
        'context': my_context,
        'rho': rho0, 'rho_u': rho_u0, 'rho_v': rho_v0, 'E': E0,
        'nx': nx, 'ny': ny,
        'dx': dx, 'dy': dy, 'dt': 0.45*dt,
        'gamma': gamma
    } 
    with Common.Timer("construct") as t:
        from GPUSimulators import EE2D_KP07_dimsplit
        importlib.reload(EE2D_KP07_dimsplit)
        sim = EE2D_KP07_dimsplit.EE2D_KP07_dimsplit(**arguments)
    print("Constructed in " + str(t.secs) + " seconds")

    #Create netcdf file and simulate
    with Common.DataDumper(outfile, mode='w', clobber=False) as outdata:
        outdata.ncfile.createDimension('time', None)
        outdata.ncfile.createDimension('x', nx)
        outdata.ncfile.createDimension('y', ny)

        outdata.time_var  = outdata.ncfile.createVariable('time', np.dtype('float32').char, 'time')
        outdata.x_var     = outdata.ncfile.createVariable('x', np.dtype('float32').char, 'x')
        outdata.y_var     = outdata.ncfile.createVariable('y', np.dtype('float32').char, 'y')
        outdata.rho_var   = outdata.ncfile.createVariable('rho', np.dtype('float32').char, ('time', 'y', 'x'), zlib=True)
        outdata.rho_u_var = outdata.ncfile.createVariable('rho_u', np.dtype('float32').char, ('time', 'y', 'x'), zlib=True)
        outdata.rho_v_var = outdata.ncfile.createVariable('rho_v', np.dtype('float32').char, ('time', 'y', 'x'), zlib=True)
        outdata.E_var     = outdata.ncfile.createVariable('E', np.dtype('float32').char, ('time', 'y', 'x'), zlib=True)

        outdata.x_var[:] = np.linspace(0, nx*dx, nx)
        outdata.y_var[:] = np.linspace(0, ny*dy, ny)

        progress_printer = Common.ProgressPrinter(n_save, print_every=10)
        for i in range(n_save+1):
            #Sanity check simulator
            sim.check()

            #Simulate
            if (i > 0):
                sim.simulate(t_end/n_save)

            #Save to file
            #rho = sim.u0[0].download(sim.stream)
            rho, rho_u, rho_v, E = sim.download()
            outdata.time_var[i] = sim.simTime()
            outdata.rho_var[i, :] = rho
            outdata.rho_u_var[i, :] = rho_u
            outdata.rho_v_var[i, :] = rho_v
            outdata.E_var[i, :] = E

            #Write progress to screen
            print_string = progress_printer.getPrintString(i)
            if (print_string):
                print(print_string)

    return outdata.filename   

In [ ]:
def genSchlieren(rho):
    #Compute length of z-component of normalized gradient vector 
    normal = np.gradient(rho) #[x, y, 1]
    length = 1.0 / np.sqrt(normal[0]**2 + normal[1]**2 + 1.0**2)
    schlieren = np.power(length, 128)
    return schlieren


def genCurl(rho, rho_u, rho_v):
    u = rho_u / rho
    v = rho_v / rho
    u = np.sqrt(u**2 + v**2)
    u_max = u.max()
    
    du_dy, du_dx = np.gradient(u)
    dv_dy, dv_dx = np.gradient(v)
    
    curl = dv_dx - du_dy
    return curl


def genColors(rho, rho_u, rho_v, cmap, vmax, vmin):
    schlieren = genSchlieren(rho)
    curl = genCurl(rho, rho_u, rho_v)
    
    colors = Normalize(vmin, vmax, clip=True)(curl)
    colors = cmap(colors)
    for k in range(3):
        colors[:,:,k] = colors[:,:,k]*schlieren
        
    return colors

In [ ]:
class VisType(Enum):
    Schlieren = 0
    Density = 1

def makeAnimation(infile, vis_type, vmin, vmax, save_anim=False, cmap=plt.cm.coolwarm, fig_args=None):
    fig = plt.figure(**fig_args)
    
    with Common.DataDumper(infile, 'r') as indata:
        time = indata.ncfile.variables['time'][:]
        x = indata.ncfile.variables['x'][:]
        y = indata.ncfile.variables['y'][:]
        rho = indata.ncfile.variables['rho'][0]
        rho_u = indata.ncfile.variables['rho_u'][0]
        rho_v = indata.ncfile.variables['rho_v'][0]
        num_frames = len(time)

        ax1 = fig.gca()
        extents = [0, x.max(), 0, y.max()]
        
        if (vis_type == VisType.Schlieren):
            im = ax1.imshow(genColors(rho, rho_u, rho_v, cmap, vmax, vmin), origin='bottom', extent=extents, cmap='gray', vmin=0.0, vmax=1.0)
            fig.suptitle("Schlieren / vorticity at t={:.2f}".format(time[0]))
        elif (vis_type == VisType.Density):
            im = ax1.imshow(rho, origin='bottom', extent=extents, cmap=cmap, vmin=vmin, vmax=vmax)
            fig.suptitle("Density at t={:.2f}".format(time[0]))
        else:
            assert False, "Wrong vis_type"

        #Create colorbar
        from mpl_toolkits.axes_grid1 import make_axes_locatable
        divider = make_axes_locatable(ax1)
        ax2 = divider.append_axes("right", size=0.1, pad=0.05)
        cb1 = ColorbarBase(ax2, cmap=cmap,
                            norm=Normalize(vmin=vmin, vmax=vmax),
                            orientation='vertical')
        
        #Label colorbar
        if (vis_type == VisType.Schlieren):
            cb1.set_label('Vorticity')
        elif (vis_type == VisType.Density):
            cb1.set_label('Density')

        progress_printer = Common.ProgressPrinter(num_frames, print_every=5)
        
        def animate(i):
            rho = indata.ncfile.variables['rho'][i]
            rho_u = indata.ncfile.variables['rho_u'][i]
            rho_v = indata.ncfile.variables['rho_v'][i]
            
            if (vis_type == VisType.Schlieren):
                im.set_data(genColors(rho, rho_u, rho_v, cmap, vmax, vmin))
                fig.suptitle("Schlieren / vorticity at t={:.2f}".format(time[i]))
            elif (vis_type == VisType.Density):
                im.set_data(rho) 
                fig.suptitle("Density at t={:.2f}".format(time[i]))
            
            #Write progress to screen
            print_string = progress_printer.getPrintString(i)
            if (print_string):
                print(print_string)

        anim = animation.FuncAnimation(fig, animate, interval=50, frames=range(num_frames))
        plt.close()

        if (save_anim):
            root, _ = os.path.splitext(infile)
            movie_outpath = os.path.abspath(root + ".mp4")
            if (os.path.isfile(movie_outpath)):
                print("Reusing previously created movie " + movie_outpath)
            else:
                print("Creating movie " + movie_outpath)
                #from matplotlib.animation import FFMpegFileWriter
                #writer = FFMpegFileWriter(fps=25)
                from matplotlib.animation import FFMpegWriter
                writer = FFMpegWriter(fps=25)
                anim.save(movie_outpath, writer=writer)
            display(HTML("""
            <div align="middle">
            <video width="80%" controls>
                <source src="{:s}" type="video/mp4">
            </video>
            </div>
            """.format(movie_outpath)))
        else:
            #plt.rcParams["animation.html"] = "html5"
            plt.rcParams["animation.html"] = "jshtml"
            display(anim)


* Liska and Wendroff, COMPARISON OF SEVERAL DIFFERENCE SCHEMES ON 1D AND 2D TEST PROBLEMS FOR THE EULER EQUATIONS, http://www-troja.fjfi.cvut.cz/~liska/CompareEuler/compare8.pdf
* https://www.astro.princeton.edu/~jstone/Athena/tests/rt/rt.html
* 

In [ ]:
nx = 301
ny = nx*3
gamma = 1.4
t_end = 30
n_save = 300
outfile = "data/euler_rayleigh-taylor.nc"
outdata = None

rho0, rho_u0, rho_v0, E0, dx, dy, dt = genRayleighTaylor(nx, ny, gamma)
outfile = makeSimulation(outfile, t_end, rho0, rho_u0, rho_v0, E0, nx, ny, dx, dy, dt, gamma)

In [ ]:
#outfile = 'data/euler_rayleigh-taylor_0007.nc'
makeAnimation(outfile, vis_type=VisType.Density, vmin=1, vmax=2, cmap=plt.cm.RdBu, save_anim=True, fig_args={'figsize':(3.4, 8), 'tight_layout':True})

In [ ]:
nx = 1600
ny = nx//4
gamma = 1.4
t_end = 0.5#3.0
n_save = 20#500
outfile = "data/euler_shock-bubble.nc"
outdata = None

rho0, rho_u0, rho_v0, E0, dx, dy, dt = genShockBubble(nx, ny, gamma)
outfile = makeSimulation(outfile, t_end, rho0, rho_u0, rho_v0, E0, nx, ny, dx, dy, dt, gamma)

In [ ]:
#outfile = 'data/euler_shock-bubble_0011.nc'
makeAnimation(outfile, vis_type=VisType.Schlieren, vmin=-0.2, vmax=0.2, cmap=plt.cm.RdBu, save_anim=True, fig_args={'figsize':(16, 5), 'tight_layout':True})

In [ ]:
nx = 1600
ny = nx//2
gamma = 1.4
roughness = 0.125
t_end = 10.0
n_save = 1000
outfile = "data/euler_kelvin_helmholtz.nc"
outdata = None

rho0, rho_u0, rho_v0, E0, dx, dy, dt = genKelvinHelmholtz(nx, ny, gamma, roughness)
outfile = makeSimulation(outfile, t_end, rho0, rho_u0, rho_v0, E0, nx, ny, dx, dy, dt, gamma)

In [ ]:
#outfile='data/euler_kelvin_helmholtz_0012.nc'
makeAnimation(outfile, vis_type=VisType.Density, vmin=1.0, vmax=2.0, save_anim=True, fig_args={'figsize':(16, 9), 'tight_layout':True})